In [ ]:
from pathlib import Path
import h5py
import matplotlib.pyplot as plt
import numpy as np


from topostats.io import hdf5_to_dict
import h5glance

In [ ]:
data_dir = Path("/Users/sylvi/topo_data/pleng/data/")
assert data_dir.exists()
filename = "20231218_2ngSCcats.0_00003.topostats"
file_path = data_dir / filename
assert file_path.exists()

with h5py.File(file_path, "r") as f:
    image_data = hdf5_to_dict(f, group_path="/")
    print(image_data.keys())
    spline_data = image_data["splining"]["above"]
    p_to_nm = image_data["pixel_to_nm_scaling"]
    print(f"pixel to nm scaling: {p_to_nm}")

    image = image_data["image"]
    plt.imshow(image)
    plt.show()

    spline_data = spline_data["grain_0"]["mol_1"]
    print(spline_data.keys())
    spline_coords = spline_data["spline_coords"]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
# for spline_coord in spline_coords:
#     plt.scatter(x=spline_coord[1], y=spline_coord[0], s=1, c="red")
# plt.scatter(x=spline_coords[0, 1], y=spline_coords[0, 0], s=10, c="blue")

ax.plot(spline_coords[:, 1], spline_coords[:, 0], c="red")
ax.scatter(spline_coords[0, 1], spline_coords[0, 0], c="blue")


# starting from the first vector, calculate the angle difference of each vector to the first one
maximum_length_nm = 10
current_length = 0
group_count = 0

distances = []
angle_diffs = []

distance_angle_diff_pairs = {}

# first vector
first_vector = spline_coords[1] - spline_coords[0]
first_angle = np.arctan2(first_vector[1], first_vector[0])
# iterate over the coordinates
for coord_index, coord in enumerate(spline_coords):
    if coord_index < 2:
        continue
    if coord_index == len(spline_coords) - 1:
        break
    vector = coord - spline_coords[coord_index - 1]
    angle = np.arctan2(vector[1], vector[0])
    angle_diff = angle - first_angle
    # print(f"angle diff: {angle_diff}")

    # calculate the length of the vector
    vector_length = np.linalg.norm(vector)
    current_length += vector_length
    if current_length > maximum_length_nm:
        # set as new first vector
        first_vector = vector
        first_angle = angle
        # Add current distance angle diffs to list
        distance_angle_diff_pairs[coord_index] = {"distances": distances, "angle_diffs": angle_diffs}
        # reset current length to start counting up again
        current_length = 0
        distances = []
        angle_diffs = []
        # plot point on spline to mark it
        ax.scatter(coord[1], coord[0], c="green")
        group_count += 1
        # if group_count == 3:
        #     break
    else:
        distances.append(current_length)
        angle_diffs.append(angle_diff)

print(distance_angle_diff_pairs.keys())

# plot angle difference as a function of distance
fig, ax = plt.subplots()
for coord_index, distance_angle_diffs_result in distance_angle_diff_pairs.items():
    angle_diffs = distance_angle_diffs_result["angle_diffs"]
    angle_diffs = np.cos(angle_diffs)
    angle_diffs = -np.log(angle_diffs)
    distances = distance_angle_diffs_result["distances"]
    ax.plot(distances, angle_diffs, label=f"point {coord_index}")
ax.legend()
plt.show()

# create an average of the angle differences despite not having the same lengths
# calculate the average of the angle differences
average_angle_diffs = []
for coord_index, distance_angle_diffs_result in distance_angle_diff_pairs.items():
    angle_diffs = distance_angle_diffs_result["angle_diffs"]
    angle_diffs = np.cos(angle_diffs)
    angle_diffs = -np.log(angle_diffs)
    distances = distance_angle_diffs_result["distances"]
    # interpolate the angle differences to a common distance
    interpolated_angle_diffs = np.interp(np.arange(0, maximum_length_nm, 0.1), distances, angle_diffs)
    average_angle_diffs.append(interpolated_angle_diffs)
average_angle_diffs = np.mean(average_angle_diffs, axis=0)
fig, ax = plt.subplots()
ax.plot(np.arange(0, maximum_length_nm, 0.1), average_angle_diffs)
plt.show()

# fit a line to the average angle differences
from scipy.optimize import curve_fit

def linear(x, a, b):
    return a * x + b

popt, pcov = curve_fit(linear, np.arange(0, maximum_length_nm, 0.1), average_angle_diffs)
print(f"popt: {popt} = 1/2p. p = {1/(2*popt[0])}")
fig, ax = plt.subplots()
ax.plot(np.arange(0, maximum_length_nm, 0.1), linear(np.arange(0, maximum_length_nm, 0.1), *popt))
ax.plot(np.arange(0, maximum_length_nm, 0.1), average_angle_diffs)
plt.show()


In [ ]:
from scipy.optimize import curve_fit


def persistence_length(points):
    # calculate tangents
    tangents = np.diff(points, axis=0)
    tangent_norms = np.linalg.norm(tangents, axis=1)
    unit_tangents = tangents / tangent_norms[:, None]

    # calculate correlation function
    distances = np.arange(1, len(points))
    # distances is the distance between the points, starting from 1 to the end of the points
    # this is used to calculate the correlation between the points at distance 1, 2, 3, etc.
    # the correlation is calculated by taking the dot product of the unit tangents at the points
    # at distance 1, 2, 3, etc. and then taking the mean of the dot products

    correlations = []
    for distance in distances:
        # for each distance, calculate the dot product of the unit tangents of the points at that distance,
        # eg: for distance 1, calculate the dot product of the unit tangents of the points at index 0 and 1

        # :-distance means all elements up to the last distance elements, eg: for distance 1, all elements up to the
        # second last element, and [distance:] means all elements starting from the distance element, eg: for distance 1,
        # all elements starting from the second element
        # so this takes the dot product
        dot_products = np.sum(unit_tangents[:-distance] * unit_tangents[distance:], axis=1)

        correlations.append(np.mean(dot_products))

    correlations = np.array(correlations)
    # remove nans
    distances = distances[~np.isnan(correlations)]
    correlations = correlations[~np.isnan(correlations)]

    print(len(correlations))

    # fit an exponential function
    def exponential_decay(x, Lp):
        return np.exp(-x / Lp)

    popt, _ = curve_fit(exponential_decay, distances, correlations)
    Lp = popt[0]

    return Lp


Lp = persistence_length(spline_coords)
print(f"persistence length: {Lp}")